In [1]:
!pip install "sympy<=1.4"

  Using cached sympy-1.4-py2.py3-none-any.whl (5.3 MB)


In [2]:
import sys
sys.path.insert(0, '/opt/app-root/src/.local/lib/python3.8/site-packages/')

In [3]:
import sympy

In [4]:
sympy.__version__

'1.4'

In [5]:
sympy.__file__

'/opt/app-root/src/.local/lib/python3.8/site-packages/sympy/__init__.py'

In [6]:
import nest                                                          # import NEST module
import matplotlib.pyplot as plt                                      # for plotting
from pynestml.frontend.pynestml_frontend import generate_nest_target # NESTML


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.6.0
 Built: Sep 28 2023 14:13:37

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [7]:
# compile nestml model (needs to be done only once)
generate_nest_target(input_path="./iaf_psc_exp.nestml",
                     target_path="./nestml_target",
                     suffix="_nestml",                     
                     logging_level='ERROR')    

# install resulting NESTML module to make models available in NEST
nest.Install('nestmlmodule') 


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.6.0
 Built: Sep 28 2023 14:13:37

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.

CMake Warning:
  Ignoring empty string ("") provided on the command line.


CMake Warning (dev) at CMakeLists.txt:93 (project):
  cmake_minimum_required() should be called prior to this top-level project()
  call.  Please see the cmake-commands(7) manual for usage documentation of
  both commands.
This warning is for project developers.  Use -Wno-dev to suppress it.

-- The CXX compiler identification is unknown
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - failed
-- Check for working CXX compiler: /srv/test-build-2309/spack/lib/spack/env/gcc/g++
-- Check for working CXX compiler: /srv/test-build-2309/spack/lib/spack/env/gcc/g++ - broke

GeneratedCodeBuildException: Error occurred during cmake! More detailed error messages can be found in stderr.

In [ ]:
nest.ResetKernel() # reset simulation kernel 

neuron=nest.Create('iaf_psc_exp_nestml') # create LIF neuron with exponential synaptic currents

# create a spike generator, and set it up to create two spikes at 10 and 30ms
spikegenerator=nest.Create('spike_generator', params={'spike_times': [10.,30.]}) 

# create multimeter and set it up to record the membrane potential V_m
multimeter=nest.Create('multimeter', {'record_from': ['V_m']})

# connect spike generator with neuron with synaptic weight 100 pA
nest.Connect(spikegenerator, neuron,syn_spec={'weight': 50.0})

nest.Connect(multimeter, neuron)  # connect multimeter to the neuron

nest.Simulate(100.) # run simulation for 100ms

# read out recording time and voltage from voltmeter
times=multimeter.get('events')['times']
voltage=multimeter.get('events')['V_m']

In [ ]:
# plot results
plt.figure(1)
plt.clf()
plt.plot(times,voltage,'k-',lw=2)
plt.xlabel('time (ms)')
plt.ylabel('membrane potential (mV)')
plt.savefig('./figures/hello_world_nestml.pdf')